In [1]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import seaborn as sns 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import scale, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report 


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:
df=pd.read_csv("./diabetes.csv")
y=df["Outcome"]
X=df.drop(['Outcome'],axis=1)
X_train, X_test,y_train, y_test=train_test_split(X,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=42)

In [65]:
scaler=StandardScaler()

In [73]:
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [75]:
scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [69]:
#model ve tahmin

In [19]:
mlpc_model = MLPClassifier().fit(X_train, y_train)

In [23]:
mlpc_model.coefs_

[array([[ 1.52443490e-01, -1.78368828e-02,  1.83696441e-02,
          8.87760906e-02, -4.47488517e-03,  2.08575692e-01,
         -2.37652765e-01, -1.81363416e-01,  1.32833534e-01,
         -2.03942716e-01,  1.06754944e-01,  1.78737495e-02,
          1.26316787e-01, -6.70925629e-02,  2.05538402e-01,
          2.12838715e-01, -1.85115801e-01,  1.00849910e-01,
         -1.00664929e-02, -1.87732343e-01,  2.43378404e-01,
          1.30508644e-01,  3.10198377e-02, -1.04083422e-01,
         -1.34307151e-01,  2.27229175e-01, -2.59307208e-02,
         -1.99230412e-01,  5.33828390e-02,  9.53460514e-02,
         -1.81260724e-01,  3.63153612e-02,  1.68749110e-01,
         -1.50279033e-01,  1.94022624e-01,  4.46843323e-02,
         -2.06627146e-01, -2.79910165e-02,  1.47687837e-01,
         -1.16460692e-01,  2.36294608e-01, -1.06678097e-02,
          1.25609507e-01,  1.72234907e-01,  1.67372654e-01,
          3.67735183e-01, -4.42236289e-02, -1.09809692e-01,
          1.49630750e-01,  1.02263310e-0

In [25]:
?mlpc_model

Type:        MLPClassifier
String form: MLPClassifier()
File:        /opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py
Docstring:  
Multi-layer Perceptron classifier.

This model optimizes the log-loss function using LBFGS or stochastic
gradient descent.

.. versionadded:: 0.18

Parameters
----------
hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
    The ith element represents the number of neurons in the ith
    hidden layer.

activation : {'identity', 'logistic', 'tanh', 'relu'}, default='relu'
    Activation function for the hidden layer.

    - 'identity', no-op activation, useful to implement linear bottleneck,
      returns f(x) = x

    - 'logistic', the logistic sigmoid function,
      returns f(x) = 1 / (1 + exp(-x)).

    - 'tanh', the hyperbolic tan function,
      returns f(x) = tanh(x).

    - 'relu', the rectified linear unit function,
      returns f(x) = max(0, x)

solver : {'lbfgs', 'sgd', 'adam'}, def

In [27]:
y_pred=mlpc_model.predict(X_test)

In [29]:
accuracy_score(y_test,y_pred)

0.7186147186147186

In [31]:
#model tuning

In [33]:
mlpc_params={"alpha":[1,2,3,5,0.1,0.01,0.005,0.0001],
             "hidden_layer_sizes":[(10,10),(100,100,100),(100,100),(3,5)]}

In [37]:
mlpc=MLPClassifier(solver="lbfgs")

In [45]:
mlpc_cv_model = GridSearchCV(estimator=mlpc_model, param_grid=mlpc_params, cv=10, n_jobs=-1, verbose=2).fit(X_train, y_train)

Fitting 10 folds for each of 32 candidates, totalling 320 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_

In [47]:
mlpc_cv_model


GridSearchCV(cv=10, estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 5, 0.1, 0.01, 0.005, 0.0001],
                         'hidden_layer_sizes': [(10, 10), (100, 100, 100),
                                                (100, 100), (3, 5)]},
             verbose=2)

In [49]:
mlpc_cv_model.best_params_

{'alpha': 5, 'hidden_layer_sizes': (100, 100, 100)}

In [51]:
#finalmodel

In [55]:
mlpc_tuned = MLPClassifier(solver="lbfgs", alpha=0.0001, hidden_layer_sizes=(3, 5)).fit(X_train, y_train)

In [57]:
y_pred=mlpc_tuned.predict(X_test)

In [59]:
accuracy_score(y_test,y_pred)

0.670995670995671